# Notebook 10: Frequency-Domain Specifications and Loop Shaping
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook explores frequency-domain specifications and loop shaping approaches for control system design, covering concepts like gain and phase margins, performance limitations, and systematic control synthesis methods.

Authors:
- Felix Muller; fmuller@ethz.ch
- Mark Benazet Castells; mbenazet@ethz.ch

## Learning Objectives

After completing this material, you should be able to:

1. Understand and specify control requirements in the frequency domain:
   - Express command tracking requirements 
   - Specify disturbance rejection requirements
   - Define noise rejection specifications

2. Use loop shaping techniques for control design:
   - Apply proportional control effectively
   - Design lead compensation for phase margin improvement
   - Design lag compensation for steady-state error reduction
   - Combine compensation elements systematically

3. Understand fundamental performance limitations:
   - Recognize the waterbed effect
   - Handle constraints from non-minimum phase zeros
   - Deal with limitations from unstable poles
   - Apply Bode's integral theorem

## Required Packages

Run the following cell to import required packages:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output, FloatSlider
from IPython.display import display, clear_output, HTML
from scipy import signal
import control as ctrl

## 1. Frequency-Domain Specifications

### 1.1 Introduction 

Frequency-domain specifications provide a powerful way to express control requirements. The key idea is to separate different control objectives by frequency:

- Low frequencies: Command tracking and disturbance rejection
- High frequencies: Noise rejection 
- Mid frequencies: Stability margins and robustness

For a closed-loop system, we work with two key transfer functions:

1. Sensitivity function: $S(s) = \frac{1}{1 + L(s)}$
   - $S(s)$ captures the system's ability to reject disturbances and determines error in tracking commands
2. Complementary sensitivity function: $T(s) = \frac{L(s)}{1 + L(s)}$
   - $T(s)$ captures the system's ability to track commands and determines the effect of noise.

where $L(s) = P(s)C(s)$ is the loop transfer function.

Key specifications are typically written as:

1. Command tracking/disturbance rejection:
   - $|S(j\omega)| \ll 1$ at low frequencies
   - Example: "ensure commands are tracked with max 10% error up to 10Hz"

2. Noise rejection:
   - $|T(j\omega)| \ll 1$ at high frequencies  
   - Example: "ensure noise is reduced by factor of 10 at frequencies above 100Hz"

### 1.3 Translating to Open-Loop Specifications
The closed-loop specifications can be translated to requirements on the open-loop transfer function $L(s)$:

1. Command tracking/disturbance rejection requirement:
   - $|S(j\omega)| \cdot |W_1(j\omega)| < 1$ 
   - This translates to $|L(j\omega)| > |W_1(j\omega)|$

2. Noise rejection requirement:
   - $|T(j\omega)| \cdot |W_2(j\omega)| < 1$
   - This translates to $|L(j\omega)| < |W_2(j\omega)|^{-1}$

3. Cross-over frequency:
   - The frequency where $|L(j\omega_{gc})| = 1$ is called the cross-over frequency $\omega_{gc}$

Where $W_1(j\omega)$ and $W_2(j\omega)$ are weighting functions that capture the desired behavior at low and high frequencies, respectively.

These create an "obstacle course" on the Bode plot through which we must navigate $L(j\omega)$. The goal would be to find a loop transfer function that stays between the two curves. 

<div style="text-align:center;">
<img src="./img/obstacle.png" alt="Obstacle" width="500">
</div>

The challenge is to design L(jω)L(j\omega)L(jω) such that:

   - It stays above W1W_1W1​ at low frequencies
   - It stays below W2−1W_2^{-1}W2−1​ at high frequencies
   - It transitions smoothly between these bounds near crossover